<a href="https://colab.research.google.com/github/nohstns/SpaceSimRep/blob/main/SimSpan_Rep_ItemGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pseudoword generation SimSpan Replication
This notebook contains the code used to generate the pseudowords used to replicate [Nakata & Suzuki (2019)](https://www.cambridge.org/core/journals/studies-in-second-language-acquisition/article/effects-of-massing-and-spacing-on-the-learning-of-semantically-related-and-unrelated-words/F58BA8D70385603B9C42E408BFCB8A10). We rely on the `Wuggy` package, the [Python implementation](https://wuggycode.github.io/wuggy/) of the algorithm developed by [Keuleers & Brysbaert (2010)](http://crr.ugent.be/papers/Wuggy_BRM.pdf).

## Loading packages


In [ ]:
!pip install Wuggy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 32.9 MB/s eta 0:00:00


In [ ]:
from wuggy import WuggyGenerator
from csv import DictWriter

In [ ]:
test_set0 = ["tximino", "azkonar", "igaraba"]

## Loading reference vocabulary

Reference (English) items

In [ ]:
ref_related_set1 = ["baboon", "badger", "otter", "porcupine", "raccoon", "weasel"]
ref_related_set2 = ["diaphragm", "intestine", "placenta", "rectum", "tympanum", "womb"]
ref_related_set3 = ["bluff", "estuary", "plateau", "ravine", "shoal", "strait"]
ref_related_set4 = ["azalea", "camellia", "camphor", "cedar", "magnolia", "willow"]

ref_unrelated_set6 = ["cistern", "insurgent", "pall", "parable", "sardine", "venom"]
ref_unrelated_set7 = ["alcove", "pail", "pigment", "potassium", "relic", "toupee"]
ref_unrelated_set8 = ["berth", "fuselage", "ointment", "ore", "sentry", "tuberculosis"]
ref_unrelated_set5 = ["alloy", "apparition", "kerosene", "kiln", "plumage", "rudder"]

english_sets = [ref_related_set1, ref_related_set2, ref_related_set3, ref_related_set4,
                ref_unrelated_set5, ref_unrelated_set6, ref_unrelated_set7, ref_unrelated_set8]

Source (real Basque words) items

In [ ]:
source_related_set1 = ["tximino", "azkonar", "igaraba", "triku", "mapatxea", "erbiarra"]
source_related_set2 = ["diafragma", "hestea", "plazenta", "ondestea", "tinpanoa", "umetokia"]
source_related_set3 = ["labarra", "estuarioa", "lautada", "arroka", "sardun-arraina", "itsasartea"]
source_related_set4 = ["azalea", "kamelia", "kanforra", "zedroa", "magnolia", "sahats"]



In [ ]:
source_unrelated_set5 = ["aleazioa", "agerpena", "kerosenoa", "labea", "lumajea", "lema"]
source_unrelated_set6 = ["zisterna", "matxinatua", "oihal-zapia", "parabola", "sardina", "pozoia"]
source_unrelated_set7 = ["alkoba", "ontzia", "pigmentua", "potasioa", "erlikia", "tupea"]
source_unrelated_set8 = ["kaiola", "fuselajea", "ukendua", "mea", "zaintzailea", "tuberkulosia"]

In [ ]:
source_sets = [source_related_set1, source_related_set2, source_related_set3, source_related_set4,
               source_unrelated_set5, source_unrelated_set6, source_unrelated_set7, source_unrelated_set8]

## Running Wuggy

In [ ]:
g = WuggyGenerator()

In [ ]:
g.supported_official_language_plugin_names

['orthographic_basque',
 'orthographic_dutch',
 'orthographic_english',
 'orthographic_french',
 'orthographic_german',
 'orthographic_italian',
 'orthographic_polish',
 'orthographic_serbian_cyrillic',
 'orthographic_serbian_latin',
 'orthographic_spanish',
 'orthographic_vietnamese',
 'orthographic_estonian',
 'phonetic_english_celex',
 'phonetic_english_cmu',
 'phonetic_french',
 'phonetic_italian']

In [ ]:
g.load('orthographic_basque')

In [ ]:
ncandidates = 3

In [ ]:
for match in g.generate_classic(test_set0):
    print(match["pseudoword"])

pribido
pribilo
prihilo
prihido
trihido
trihilo
tribilo
tribido
fribilo
fribido
azpudar
azpular
aztudar
aztular
askular
askudar
aspolar
aspodar
aspunar
aktolar
omalada
omalafa
omalaja
omalasa
omalapa
omalaxa
omalama
omalaza
omalaha
omalaga


Steps to add:

- zip generated pseudoword with original English word
- structure export so it has the columns `Item - Set - English - Pseudo - Reference`

In [ ]:
pseudoword_matches = g.generate_classic(test_set0)
g.export_classic_pseudoword_matches_to_csv(pseudoword_matches, "./pseudowords.csv")